In [1]:
import numpy as np
import pandas as pd
import math
%matplotlib inline
from fastai.imports import *
from fastai.structured import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics

In [40]:
class TreeEnsemble():
    def __init__(self,x,y,n_trees,sample_sz,min_leaf=5, bootstrap = False, random_state = None, max_features = 1):
        #the seed will be set for a constant for now to be able to compare with sklearn tree
        if random_state != None:
            np.random.seed(random_state)
        self.x,self.y,self.sample_sz,self.min_leaf,self.bootstrap, = x,y,sample_sz,min_leaf,bootstrap
        self.max_features = max_features
        #here we create trees by calling a function create tree for the required number of trees
        self.trees = [self.create_tree() for i in range(n_trees)]
    def create_tree(self):
        if self.bootstrap == True:
            idxs = []
            for i in range(self.sample_sz):
                idxs.append(np.random.randint(0,len(self.y)))
            idxs = np.array(idxs)
        else:
            idxs = np.random.permutation(len(self.y))[:self.sample_sz]
        #iloc[row,column] or iloc[[list of rows],[list of columns]] .. columns are optional 
        return DecisionTree(self.x.iloc[idxs],self.y[idxs],min_leaf = self.min_leaf, max_features = self.max_features)
#         return idxs
    def predict(self,x):
        #the below is not a recursive function the predict below will be done for a decision tree
        #we will calculate the prediction of each tree for given x (features) and average using np.mean
        #axis = 0 will return the average of matrix rows
        return np.mean([tree.predict(x) for tree in self.trees],axis = 0)
    @property
    def _feature_importance(self):
        fi = []
        for i in range(self.x.shape[1]):
            shuf = self.x.iloc[:,i].copy()
            self.x.iloc[:,i] = np.random.permutation(self.x.iloc[:,i])
            fi.append(rmse(self.predict(self.x.values),self.y))
            self.x.iloc[:,i] = shuf
        return np.array(fi)

In [41]:
class DecisionTree():
    #the indices are there to keep track of which of the row indixes went to the left and the right side of the tree
    def __init__ (self,x,y,idxs = None,min_leaf = 5, max_features = 1):
        # y passed to the decision tree will be the random selection of the treeensemble
        if idxs is None: idxs = np.arange(len(y))
        self.x, self.y, self.idxs, self.min_leaf = x,y,idxs,min_leaf
        # we need to keep track of the number of rows and number of columns
        self.n,self.c = len(idxs), x.shape[1]
        self.max_features = max_features
        #every node of the tree will have a value(prediction) which is equal to the mean of the node indexes
        self.val = np.mean(y[idxs])
        #some nodes of the tree will have a score which is how effective is the split only if it is not a LEAF NODE
        #at the begining we have not created any splits yet so the score will be infinity
        self.score = float('inf')
        #the next step is to determine which variable we will split on and what level we will split on
        self.find_varsplit()
        
    def find_varsplit(self):
        # max features will be a number from 0 to 1 randomly select a ratio of the columns to look for the next split
        #we will create a random list of all columns indexes and return only the ratio determined by max_features
        for i in np.random.permutation(range(self.c))[:int(self.max_features*self.c)]:
            self.find_better_split(i)
        if self.is_leaf: return
        x = self.split_col
        lhs = np.nonzero(x<=self.split)[0]
        rhs = np.nonzero(x>self.split)[0]
        self.lhs = DecisionTree(self.x,self.y,self.idxs[lhs])
        self.rhs = DecisionTree(self.x,self.y,self.idxs[rhs])

    #another method to calculate the standard deviation is square root of (the mean of squares minues square of the mean)
    #the function takes the count to calculate the means, sum of values and sum of squared values.
#     def std_agg(cnt,s1,s2): return math.sqrt((s2/cnt)  - (s1/cnt)**2)
    
    def find_better_split(self,var_idx):
        #x will be all the value in a specific column at certain indexes (when the tree splits the indexes will change so
        #we need to mark the idexes for the next split
        #y is the value of the indexes
        
        x,y = self.x.values[self.idxs,var_idx],self.y[self.idxs]
        
        #we will sort all the values based on X
        sort_idx = np.argsort(x)
        #create a sorted x and y based on the sorting indexes from argsort
        x_sorted,y_sorted = x[sort_idx],y[sort_idx]
        #the RHS will be initialzed with all value (count, sums and sums squares)
        rhs_cnt,rhs_sum,rhs_sum2 = self.n, y_sorted.sum(),(y_sorted**2).sum()
        #the LHS will be empty and we will start moving a value one by one and caluclate the standard deviation
        lhs_cnt,lhs_sum,lhs_sum2 = 0,0.,0.
        
        #now we will loop through each index in x up to the min leaf (to avoid having values less than min leaf in a side)
        
        for i in range(0,self.n-self.min_leaf-1):
            xi , yi = x_sorted[i],y_sorted[i]
            lhs_cnt +=1 ; rhs_cnt -=1
            lhs_sum += yi; rhs_sum -=yi
            lhs_sum2 += yi**2 ; rhs_sum2 -= yi**2
            
            # we have to check that i is more than min sample leaf (to avoid having values less than min leaf in a side)
            #we also have to check that the next index is not equal to include every similar number in each trial
            if i<self.min_leaf or xi == x_sorted[i+1]: continue
            
            #now we will calculate the std for each split trial
            
            lhs_std = std_agg(lhs_cnt,lhs_sum,lhs_sum2); rhs_std = std_agg(rhs_cnt,rhs_sum,rhs_sum2)
            
            #now we will calculate the weighted average std of RHS and LHS and compare to the best available split score
            
            curr_score = lhs_std*lhs_cnt + rhs_std*rhs_cnt
            
            if curr_score<self.score:
                self.var_idx,self.score,self.split = var_idx,curr_score, xi
            
    @property
    def split_name(self): return self.x.columns[self.var_idx]
    
    @property
    def split_col(self): return self.x.values[self.idxs,self.var_idx]
    
    @property
    def is_leaf(self): return self.score == float('inf')
    
    def __repr__(self):
        s = f'n: {self.n}; val: {self.val}'
        if not self.is_leaf:
            s += f'; score:{self.score}; split:{self.split}; var:{self.split_name}'
        return s
    
    def predict(self, x):
        return np.array([self.predict_row(xi) for xi in x])

    def predict_row(self, xi):
        if self.is_leaf: return self.val
        t = self.lhs if xi[self.var_idx]<=self.split else self.rhs
        return t.predict_row(xi)

In [42]:
def std_agg(cnt,s1,s2): return math.sqrt((s2/cnt)  - (s1/cnt)**2)
def rmse(x,y): return math.sqrt(((x-y)**2).mean())

In [43]:
PATH = "data/bulldozers/"

df_raw = pd.read_feather('bulldozers-raw')
df_trn, y_trn, nas = proc_df(df_raw, 'SalePrice')

In [44]:
def split_vals(a,n): return a[:n], a[n:]
n_valid = 12000
n_trn = len(df_trn)-n_valid
X_train, X_valid = split_vals(df_trn, n_trn)
y_train, y_valid = split_vals(y_trn, n_trn)
raw_train, raw_valid = split_vals(df_raw, n_trn)

In [45]:
cols = ['MachineID', 'YearMade', 'MachineHoursCurrentMeter', 'ProductSize', 'Enclosure',
        'Coupler_System', 'saleYear']

In [46]:
%time tree = TreeEnsemble(X_train[cols], y_train, 5, 1000,random_state=42,bootstrap=True,max_features=0.7, min_leaf=1)

Wall time: 1.42 s


In [50]:
t = tree._feature_importance

In [60]:
t

array([0.42732, 0.62374, 0.41404, 0.59497, 0.47998, 0.55549, 0.46935])

In [77]:
total = t.sum();total

17.824507886915626

In [79]:
for i in t:
    print(i/total)

0.1198694189259907
0.17496626829856407
0.11614296517601605
0.16689791486172398
0.13464093171789607
0.15582336491582904
0.13165913610398


In [76]:
m.feature_importances_

array([0.15248, 0.24406, 0.01905, 0.20356, 0.0577 , 0.20396, 0.11919])

In [48]:
%time preds = tree.predict(X_valid[cols].values)

Wall time: 505 ms


In [780]:
metrics.r2_score(y_valid, preds)

0.712003540865145

In [54]:
ens = TreeEnsemble(X_train[cols], y_train, n_trees= 5, sample_sz = 1000, random_state=42)
tree = ens.trees[0]
x_samp,y_samp = tree.x, tree.y
x_samp.columns

Index(['MachineID', 'YearMade', 'MachineHoursCurrentMeter', 'ProductSize',
       'Enclosure', 'Coupler_System', 'saleYear'],
      dtype='object')

In [70]:
from mlsupportmm import set_rf_samples , reset_rf_samples

In [71]:
set_rf_samples(1000)

In [74]:
m = RandomForestRegressor(n_estimators=5, bootstrap=True, random_state = 42)
m.fit(X_train[cols], y_train)
# draw_tree(m.estimators_[0], x_samp, precision=2)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
                      oob_score=False, random_state=42, verbose=0,
                      warm_start=False)

In [75]:
m.feature_importances_

array([0.15248, 0.24406, 0.01905, 0.20356, 0.0577 , 0.20396, 0.11919])

In [469]:
ens.trees[0]

n: 1000; val: 10.079014121552744; score:658.5510186055565; split:1974.0; var:YearMade

In [ ]:
np.nonzero()

In [335]:
testset  = [1,5,6,2,4]

In [337]:
srt = np.argsort(testset)

In [339]:
srt

array([0, 3, 4, 1, 2], dtype=int64)

In [340]:
np.array(testset)[srt]

array([1, 2, 4, 5, 6])

In [321]:
boston = datasets.load_boston()

In [322]:
df = pd.DataFrame(data = boston.data ,columns = boston.feature_names)

In [323]:
X = df.drop('LSTAT',1)
y = df.LSTAT

In [324]:
m = TreeEnsemble(X,y,10,100)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\series.py:1143: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


In [334]:
x = X.values[:100,2]

xi = 2.31

x = x>xi;x


array([False,  True,  True, False, False, False,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False, False, False,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [325]:
m.trees[0]

n: 100; val: 10.874782608695652

In [193]:
m.create_tree()

array([459, 247, 360, 399, 388, 352, 300, 497, 171, 261, 446,  18,  77,
        99, 274, 382, 156,  13, 471,  23, 158, 196, 174,  32, 447, 392,
       231, 316,  15, 409, 344, 172,  40, 427, 256, 435, 303, 271, 491,
        73,  21, 395, 227,  91, 259, 120, 346, 239,  68, 335, 273, 243,
       472, 161, 122, 310, 240, 123, 153, 370, 213,  50, 389,  79, 481,
       260, 381, 503, 192, 194, 145, 384, 134, 251,  60, 282, 286, 141,
       366, 193, 132, 425,  22,  90, 323, 375, 121,  69, 466, 182, 403,
       140,  45,  78, 266, 152, 169, 474,  61, 483])

In [209]:
len(m.create_tree())

100

In [314]:
m = TreeEnsemble(X,y,10,100,bootstrap = True, random_state=42)

In [315]:
try1 = m.create_tree()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
       85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])

In [208]:
len(set(try1))

94

In [156]:
m.create_tree()

array([ 43, 136, 420,  73,  35, 100, 475, 155, 425, 254, 170, 468, 215,
       169, 251, 359, 321,  54, 455, 431,  90, 277, 100, 154, 376, 348,
       450, 363, 140, 172,   9, 227, 264, 437, 247, 246, 154,  33, 498,
       128, 159,  88,  92, 326, 240, 322, 467, 403, 116,  93, 162,  98,
       112, 358,  90, 345, 205,  16, 273, 172, 115, 348, 467, 180,  90,
        81, 323, 448, 113, 256,  57, 153, 364, 403, 115, 342, 265, 314,
       202,  40, 104, 388, 377, 192, 400, 277, 353, 503, 236, 424, 220,
        67, 196, 255, 138, 311, 434, 148,  98,  75, 281, 370, 402, 108,
       187, 198, 302, 502, 326, 153,  39, 132, 404, 461, 131, 278, 361,
        83, 398, 241, 115, 108, 220, 391,  40, 367, 146, 111, 102,  45,
       411, 493, 504, 474, 323, 377, 270, 482, 420, 384, 194, 366, 279,
       282,  39, 446, 188, 163, 481, 476,  23,  23, 489,   3, 322, 289,
       440, 432, 470, 180, 247, 115, 278, 364,  68, 350, 253, 180, 317,
       168, 312, 248, 187,  53, 325, 356, 312, 137, 167, 390, 10

In [212]:
inds = np.random.permutation(len(y))[:100]; inds

array([126, 207, 298, 415, 232,  48, 433, 330, 412, 134, 432, 236, 396,
       208, 241, 446, 296,  93, 465, 406, 381, 435, 161, 502, 127, 256,
       410, 202, 150,  42, 319, 496,  29, 386, 462, 397, 326, 442,  75,
       417,   9, 391,  84, 254, 471, 360,  82, 457, 447, 453, 437, 120,
       458,  50, 340, 305, 331,  40,  65, 333, 178, 312, 210, 203, 324,
       195, 368, 109, 131, 237, 316, 430, 253, 179,  86, 416, 479, 407,
       156, 320,  77, 264, 341,  94, 439, 211, 213, 111, 173, 428, 343,
       262,  70, 183, 380, 105, 313,   7,  52, 499])

In [ ]:
#iloc[row,column] or iloc[[list of rows],[list of columns]] .. columns are optional 

In [127]:
from sklearn.ensemble import RandomForestRegressor

In [129]:
??RandomForestRegressor

In [218]:
for i in np.random.permutation(range(10))[:int(1*len(range(10)))]:
    print(i)

0
5
9
1
3
2
8
4
6
7


In [302]:
ids = np.arange((len(y)))

In [303]:
np.mean(y[ids])

12.653063241106723